<img src="data.table.jpg">

<br><br><br><br>
## 1. Create a simple table
---

In [ ]:

library(data.table)

# A simple data table
DT <- data.table(MemberID = 1:4,
                 FirstName = c('Alice', 'Brad', 'Chris', 'Donna'),
                 LastName = c('Allen', 'Boyle', 'Coen', 'Davis'),
                 DateJoined = as.Date(c('2017-03-25', '2019-11-04', '2018-09-02', '2017-02-12')),
                 Position = factor(c('Organizer', 'Organizer', 'Director', 'Treasurer')))
DT


In [ ]:

# Display DT column classes
sapply(DT, class)


In [ ]:

# Compare to a data frame
DF <- data.frame(MemberID = 1:4,
                 FirstName = c('Alice', 'Brad', 'Chris', 'Donna'),
                 LastName = c('Allen', 'Boyle', 'Coen', 'Davis'),
                 DateJoined = as.Date(c('2017-03-25', '2019-11-04', '2018-09-02', '2017-02-12')),
                 Position = factor(c('Organizer', 'Organizer', 'Director', 'Treasurer')))

sapply(DF, class)


In [ ]:

# Class 'data.table' extends class 'data.frame'
class(DT)


<br><br><br><br>
## 2. Read/write a file
---

In [ ]:

# Read a 14 MB .csv file into a data frame using read.csv()
system.time(
    ratings_df <- read.csv('nbaallelo.csv')
)[3]

# Source: https://github.com/fivethirtyeight/data/tree/master/nba-elo


In [ ]:

# Read a 14 MB .csv file into a data table using fread() 
system.time(
    ratings <- fread('nbaallelo.csv')
)[3]


In [ ]:
?fread


In [ ]:

# Write a data frame using write.csv()
system.time(
    write.csv(ratings_df, 'ratings_df.csv')
)[3]


In [ ]:

# Write a data table using fwrite()
system.time(
    fwrite(ratings, 'ratings.csv')
)[3]


In [ ]:

ratings


In [ ]:
nrow(ratings
    )

<br><br><br><br>
## 3. Subset and select
---

Basic syntax:

<img src="basic syntax.jpg">

In [ ]:

# Subset only rows (observations) involving the New York Knicks
rowSubset <- ratings[team_id == 'NYK' | opp_id == 'NYK']
rowSubset


In [ ]:

# Select specified columns using character vector (method 1)
columnSubset <- ratings[, c('date_game', 'team_id', 'pts', 'opp_id', 'opp_pts', 'forecast')]
columnSubset


In [ ]:

# Select specified columns using vector of variable names (method 2)
varNames <- c('date_game', 'team_id', 'pts', 'opp_id', 'opp_pts', 'forecast')

# This doesn't work!
columnSubset <- ratings[, varNames]

# ratings.dt doesn't contain a column named 'varNames'


In [ ]:

# This *does* work
columnSubset <- ratings[, ..varNames]
columnSubset


In [ ]:

# Drop specified columns using character vector of variable names
columnDropSubset <- ratings[, !c('date_game', 'team_id', 'pts', 'opp_id', 'opp_pts', 'forecast')]
columnDropSubset


In [ ]:

# Select multiple columns (variables) using list of variables
columnSubset <- ratings[, .(date_game, team_id, pts, opp_id, opp_pts, forecast)]
columnSubset


In [ ]:

# Subset and rename columns using list specification
renamedColumnSubset <-
  ratings[, .(date = date_game, team_id, team_pts = pts, opp_id, opp_pts, team_forecast = forecast)]
renamedColumnSubset


In [ ]:

# Subset rows and columns
rowColumnSubset <- ratings[team_id == 'NYK' | opp_id == 'NYK',
                                 .(date_game, team_id, pts, opp_id, opp_pts, team_forecast = forecast)]
rowColumnSubset


In [ ]:

# Calculate median forecast for Knicks in 1972-1973 season
medianFcst <- ratings[team_id == 'NYK' & year_id == 1973,
                         list(median_fcst = median(forecast), sd_fcst = sd(forecast))]
medianFcst


In [ ]:

# Calculate median forecast for Knicks, grouped by year
medianByYear <- ratings[team_id == 'NYK',
                              .(median_fcst = median(forecast)),
                              by = .(year = year_id)]
medianByYear


In [ ]:

# Calculate median forecast and mean points scored for Knicks, grouped by year and game location
medAndMeanByYearAndLocation <- ratings[team_id == 'NYK',
                                             .(median_fcst = median(forecast), mean_pts = mean(pts)),
                                             by = .(year_id, game_location)]
medAndMeanByYearAndLocation


<br><br><br><br>
## 4. Special symbols in j
---

In [ ]:

# Determine the number of observations in which the Knicks played the Lakers, grouped by year
gamesByYear <- ratings[team_id == 'NYK' & opp_id == 'LAL',
                             .N,
                             by = .(year_id)]
gamesByYear


In [ ]:

# Determine the row indices of observations in which the Knicks played the Lakers, grouped by year
rowIndices <- ratings[team_id == 'NYK' & opp_id == 'LAL',
                            .(.I),
                            by = .(year_id)]
rowIndices


In [ ]:

# View the man page for special symbols
ratings[3, 4]
ratings$year_id[4]

<br><br><br><br>
## 5. Update tables by reference
---

In [ ]:

# Coerce a data frame to a data table by making a copy
dataFrameToDataTable <- as.data.table(DF)

class(DF)
class(dataFrameToDataTable)


In [ ]:

# Coerce a data frame to a data table by reference
setDT(DF)

class(DF)


In [ ]:

DT

# Add a column using :=
DT[, FullName := paste(FirstName, LastName)]

DT


In [ ]:

# Add multiple columns
DT[, c('MonthJoined', 'DayJoined', 'YearJoined') :=
   .(format(DateJoined, '%m'), format(DateJoined, '%d'), format(DateJoined, '%Y'))]

DT


In [ ]:

# Drop columns
DT[, c('FirstName', 'LastName') := NULL]

DT


In [ ]:

# Sort by increasing value of DateJoined
setorder(DT, DateJoined)
DT


In [ ]:

# Sort by increasing Position and decreasng FullName
setorder(DT, Position, -FullName)
DT


In [ ]:

# Reorder columns
setcolorder(DT, c('YearJoined', 'MonthJoined', 'DayJoined'))

DT


<br><br><br><br>
## 6. Merge tables (a different syntax)
---

In [ ]:

# Read box score table from file
hou2015 <- fread('box_score_houston_rockets_2014-2015.csv')
hou2015

# Source:  https://stats.nba.com


In [ ]:

# SQL equivalent:
#
#  SELECT *
#  FROM hou2015
#  RIGHT JOIN ratings
#  ON hou2015.TEAM = ratings.team_id
#  AND hou2015.DATE = ratings.date_game
#

# Using data table merge syntax
hou2015_rj_ratings <- hou2015[ratings, on = c('TEAM' = 'team_id',  'DATE' = 'date_game')]

# Equivalent use of merge():
#
#  hou2015_rj_ratings <- merge(x = ratings, y = hou2015,
#                              by.x = c('team_id', 'date_game'), by.y = c('TEAM', 'DATE'),
#                              all.x = TRUE, all.y = FALSE)
#

hou2015_rj_ratings


In [ ]:

hou2015_rj_ratings[, .N]

ratings[, .N]


In [ ]:

# SQL equivalent:
#
#  SELECT *
#  FROM hou2015
#  LEFT JOIN ratings
#  ON hou2015.TEAM = ratings.team_id
#  AND hou2015.DATE = ratings.date_game
#

# Using data table merge syntax
hou2015_lj_ratings <- ratings[hou2015, on = c('team_id' = 'TEAM' ,  'date_game' = 'DATE')]

# Equivalent use of merge():
#
#  hou2015_lj_ratings <- merge(x = ratings, y = hou2015,
#                              by.x = c('team_id', 'date_game'), by.y = c('TEAM', 'DATE'),
#                              all.x = FALSE, all.y = TRUE)
#

hou2015_lj_ratings


In [ ]:

hou2015_lj_ratings[, .N]

hou2015[, .N]


In [ ]:

# SQL equivalent:
#
#  SELECT *
#  FROM hou2015
#  INNER JOIN ratings
#  ON hou2015.TEAM = ratings.team_id
#  AND hou2015.DATE = ratings.date_game
#

hou2015_ij_ratings <- ratings[hou2015, on = c("team_id" = "TEAM" ,  'date_game' = 'DATE'), nomatch = 0]

# Equivalent use of merge():
#
#  hou2015_ij_ratings <- merge(x = ratings, y = hou2015,
#                              by.x = c('team_id', 'date_game'), by.y = c('TEAM', 'DATE'),
#                              all.x = FALSE, all.y = FALSE)
#

hou2015_ij_ratings


In [ ]:

hou2015_ij_ratings[, .N]

hou2015[, .N]

ratings[, .N]


<br><br><br>
## Thank you!
<br><br>
<img src="hop in.jpg">